## *Using neural networks to predict and classify crystal structures of elements*

Grade 25 points

Packages

<ul>
    <li> Pymatgen </li>
    <li> Mendeleev </li>
    <li> Keras </li>
    <li> Pandas </li>
    </ul>
    
    

**Why?** Neural networks are widely used for image classification, learning structures and substructures within the data to identify patterns. Such neural network based classifiers can identify patterns and correlations within the data.

**What?** In this tutorial we will learn how to use Neural Networks to create a Classification Model to estimate the ground state of the crystal structure. <br>

In Lab-5, we performed logistic regression to predict crystal structure for a number of elements. We are going to perform the same exercise but with neural networks. We are also going to use a powerful neural network library keras to create and fit the neural network. Neural network provided by SKLearn does good job in fitting but is limited in functionality. eg. we can't modify the loss function to include regularizations terms. To avoid overfittin, a very powerful technique known as dropout is used by Neural network community. This functionality is missing in the current implementation of SKLearn. For these reasons, we will switch to keras to train our Neural networks.

### 1. Getting a dataset

In this section we will query both [Pymatgen](http://pymatgen.org/) and [Mendeleev](https://mendeleev.readthedocs.io/en/stable/) to get a complete set of properties per element. We will use this data to create the cases from which the model will train and test.
<br>
<br>
In this first snippet of code we will import all relevant libraries, the elements that will be turned into cases and the properties that will serve as the attributes for the cases. We will get 49 entries (which is a small dataset), but should give us a somewhat accurate prediction. We will also include some values to "patch" some unknown values in the dataset. It is important to note that more entries would move the prediction closer to the real value, and so would more attributes.
<br>
<br>
The elements listed were chosen because querying them for these properties yields a dataset with few unknown values, and because they represent the three most common crystallographic structures.

In [ ]:
# !pip install pymatgen
# !pip install mendeleev

In [3]:
import pymatgen as pymat
import mendeleev as mendel
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [163]:
fcc_elements = ["Ag", "Al", "Au", "Cu", "Ir", "Ni", "Pb", "Pd", "Pt", "Rh", "Th", "Yb"]
bcc_elements = ["Ba", "Cr", "Cs", "Eu", "Fe", "Li", "Mn", "Mo", "Na", "Nb", "Rb", "Ta", "V", "W" ]
hcp_elements = ["Be", "Ca", "Cd", "Co", "Dy", "Er", "Gd", "Hf", "Ho", "Lu", "Mg", "Re", 
                "Ru", "Sc", "Tb", "Ti", "Tl", "Tm", "Y", "Zn", "Zr"]
#others = ["Si", "Ge"] # "Si" and "Ge" are Face-centered diamond-cubic;


# This is the list containing all the above elements
elements = fcc_elements  + bcc_elements + hcp_elements

# Below is the list of features that we need to obtain from Mendeleev database
querable_mendeleev = ["atomic_number", "atomic_volume", "boiling_point",
                      "en_ghosh",  "evaporation_heat", "heat_of_formation",
                     "lattice_constant", "specific_heat"]

# Below is the list of features that we need to obtain from Pymaten database
querable_pymatgen = ["atomic_mass", "atomic_radius", "electrical_resistivity",
                     "molar_volume", "bulk_modulus", "youngs_modulus",
                     "average_ionic_radius", "density_of_solid",
                     "coefficient_of_linear_thermal_expansion"]

#The list below includes all the properties
querable_values = querable_mendeleev + querable_pymatgen

After setting these values, we will proceed with our queries. Depending on the database (either Pymatgen or Mendeleev) where the property can be found, the code below fills up a list with the properties of each of the elements. To visualize how the dataset we just created looks, we will use the [Pandas](https://pandas.pydata.org/) library to display it. This library will take the list of lists and show it in a nice, user-friendly table with the properties as the column headers.

* <font color=blue> **Exercise 1. (2 points)** Fill in the code snippet below to query Pymatgen and Mendeleev for the required properties.</font>

In [164]:
all_values = [] # Values for Attributes
all_labels = [] # Crystal structure (Property to be estimated)

for item in elements:
    element_values = []
    
    # This section queries Mendeleev
    mend = mendel.element(item)
    for query in querable_mendeleev:
      element_values.append(eval("mend." + query))


    # This section queries Pymatgen
    pyma = pymat.Element(item)
    for query in querable_pymatgen:
      element_values.append(eval("pyma." + query))
        
    all_values.append(element_values) # All lists are appended to another list, creating a list of lists


In [165]:
# Only run this cell once you are done with data query part in the cell above.
# This cell is for debugging purpose. 
assert(len(all_values) == len(elements)),"Len of all_values not equal to len of elements."
len(elements)

47

### One-hot encoding

Under binary classification, we assign the value of 1 to first class and 0 to the second class. To handle the multi-class classificiation problems, we employ a method known as one-hot encoding. For each sample, we assign a vector instead of a single value. This vector has dimensionality equal to the number of classes in the dataset. All the enteries are zero except the one corresponding to its class. 

As an example, in this lab we are classifying elements into 'FCC', 'BCC' or 'HCP. So there are 3 classes in this dataset. For each input sample we will assign a output vector or class label of dimensionality 3. An element with 'FCC' crystal can be assigned as [1,0,0]. An element with 'BCC' crystal can have class label as [0,1,0] and 'HCP' will be represented as [0,0,1].

* <font color=blue> **Exercise 2. (2 points)** Convert outY into class labels using one-hot encoding as explained above
.</font>

In [170]:
#Ques 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
all_labels = []

for item in elements:
  if item in fcc_elements:
    all_labels.append(np.array([1, 0, 0]).astype(int))
  elif item in bcc_elements:
    all_labels.append(np.array([0, 1, 0]).astype(int))
  elif item in hcp_elements:
    all_labels.append(np.array([0, 0, 1]).astype(int))
    
all_labels = np.array(all_labels)
all_labels.shape

(47, 3)

In [171]:
# This code replaces some missing values in the data. You don't need to fill anything here. Just run this cell before moving forward.
# Pandas Dataframe
df = pd.DataFrame(all_values, columns=querable_values)

# We will patch some of the values that are not available in the datasets.

# Value for the CTE of Cesium
index_Cs = df.index[df['atomic_number'] == 55]
df.iloc[index_Cs, df.columns.get_loc("coefficient_of_linear_thermal_expansion")] = 0.000097 
# Value from: David R. Lide (ed), CRC Handbook of Chemistry and Physics, 84th Edition. CRC Press. Boca Raton, Florida, 2003

# Value for the CTE of Rubidium
index_Rb = df.index[df['atomic_number'] == 37]
df.iloc[index_Rb, df.columns.get_loc("coefficient_of_linear_thermal_expansion")] = 0.000090 
# Value from: https://www.azom.com/article.aspx?ArticleID=1834

# Value for the Evaporation Heat of Ruthenium
index_Ru = df.index[df['atomic_number'] == 44]
df.iloc[index_Ru, df.columns.get_loc("evaporation_heat")] = 595 # kJ/mol 
# Value from: https://www.webelements.com/ruthenium/thermochemistry.html

# Value for the Bulk Modulus of Zirconium
index_Zr = df.index[df['atomic_number'] == 40]
df.iloc[index_Zr, df.columns.get_loc("bulk_modulus")] = 94 # GPa 
# Value from: https://materialsproject.org/materials/mp-131/
# Value for the Bulk Modulus of Germanium
index_Ge = df.index[df['atomic_number'] == 32]
df.iloc[index_Ge, df.columns.get_loc("bulk_modulus")] = 77.2 # GPa 
# Value from: https://www.crystran.co.uk/optical-materials/germanium-ge

# Value for the Young's Modulus of Germanium
index_Ge = df.index[df['atomic_number'] == 32]
df.iloc[index_Ge, df.columns.get_loc("youngs_modulus")] = 102.7 # GPa 
# Value from: https://www.crystran.co.uk/optical-materials/germanium-ge

df.head(n=10)

,atomic_number,atomic_volume,boiling_point,en_ghosh,evaporation_heat,heat_of_formation,lattice_constant,specific_heat,atomic_mass,atomic_radius,electrical_resistivity,molar_volume,bulk_modulus,youngs_modulus,average_ionic_radius,density_of_solid,coefficient_of_linear_thermal_expansion
0,47,10.30,2485.0,0.147217,254.1,284.9,4.09,0.237,107.868200,1.60,1.630000e-08,10.27,100.0,83.0,1.086667,10490.0,0.000019
1,13,10.00,2740.0,0.150078,284.1,330.9,4.05,0.900,26.981539,1.25,2.700000e-08,10.00,76.0,70.0,0.675000,2700.0,0.000023
2,79,10.20,3080.0,0.261370,340.0,368.2,4.08,0.129,196.966569,1.35,2.200000e-08,10.21,220.0,78.0,1.070000,19300.0,0.000014
3,29,7.10,2840.0,0.151172,304.6,337.4,3.61,0.385,63.546000,1.35,1.720000e-08,7.11,140.0,130.0,0.820000,8920.0,0.000017
4,77,8.54,4403.0,0.251060,604.0,669.0,3.84,0.133,192.217000,1.35,4.700000e-08,8.52,320.0,528.0,0.765000,22650.0,0.000006
5,28,6.60,3005.0,0.147207,378.6,430.1,3.52,0.443,58.693400,1.35,7.200000e-08,6.59,180.0,200.0,0.740000,8908.0,0.000013
6,82,18.30,2013.0,0.177911,177.8,195.2,4.95,0.159,207.200000,1.80,2.100000e-07,18.26,46.0,16.0,1.122500,11340.0,0.000029
7,46,8.90,3413.0,0.144028,372.4,376.6,3.89,0.244,106.420000,1.40,1.080000e-07,8.56,180.0,121.0,0.846250,12023.0,0.000012
8,78,9.10,4100.0,0.256910,470.0,565.7,3.92,0.133,195.084000,1.35,1.060000e-07,9.09,230.0,168.0,0.805000,21090.0,0.000009
9,45,8.30,4000.0,0.140838,494.0,556.0,3.80,0.244,102.905500,1.35,4.300000e-08,8.28,380.0,275.0,0.745000,12450.0,0.000008


### 2. Processing and Organizing Data

We again normalize the data and organize it into training and testing sets as before.

##### SETS

We have 47 elements for which the crystal structure is known and we will use 40 of these as a training set and the remaining 7 as testing set.

##### NORMALIZATION

We will again use the Standard Score Normalization, which subtracts the mean of the feature and divide by its standard deviation.

<span style="font-size:2em;">$ \frac{X - µ}{σ} $ </span>

While our model might converge without feature normalization, the resultant model would be difficult to train and would be dependent on the choice of units used in the input.

* <font color=blue> **Exercise 4. (3 points)** Perform normalization on the input features stored in the dataframe df </font>

In [172]:
normX = (df-df.mean())/df.std()


* <font color=blue> **Exercise 5. (1 point)** Check if there is any Nan in the dataframe normX </font> 

In [173]:
normX.isnull().values.any()

False

##### Split data into training and test set

* <font color=blue> **Exercise 6. (1 point)** Use SKLearn's train_test_split to obtain training and test set. Use 20% as test set. Output are stored in all_labels variable. </font>

In [177]:
from sklearn.model_selection import train_test_split

all_labels = train_test_split(all_labels, test_size=0.2, random_state=42)
vals = train_test_split(normX.values, test_size=0.2, random_state=42)
df_train = vals[0]
df_test = vals[1]


### 3. Creating the Model

For this classification, we will use a simple sequential neural network with one densely connected hidden layer. The optimizer used will be [RMSPropOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer) (Root Mean Square Propagation).

To learn more about Root Mean Squared Propagation, click [here](https://climin.readthedocs.io/en/latest/rmsprop.html).



The key difference between the regression model and the classification model is our metric to measure network performance. While we used mean squared error (between the true outputs and the network's predicted output) for the regression task, we use categorical crossentropy (click [here](https://towardsdatascience.com/demystifying-cross-entropy-e80e3ad54a8) to learn more about it), using classification accuracy as a metric where higher accuracy implies a better network.

* <font color=blue>  In this section we will learn about a powerful python library, keras. This library is easy to use and can fit different neural networks. There are number of tutorials online to help you start with Keras library and we highly recommend to use these sources to finish this lab. </font>

* <font color=blue> **Exercise 7. (1 point)** Import keras library and define a sequential model.  </font>

In [84]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()


* <font color=blue> **Exercise 8. (1 point)** Add a dense layer to your model with 10 hidden units. The activation function is 'relu'.  </font>

In [85]:
model.add(layers.Dense(10, activation='relu'))

* <font color=blue> **Exercise 9. (1 point)** Compile your model with 'categorical_crossentropy' loss, 'RMSProp' as optimizer and 'accuracy' as metrics. </font>

In [86]:
model.compile(optimizer='RMSProp', loss='categorical_crossentropy')

* <font color=blue> **Exercise 10. (1 point)** Fit your model on training set </font>

In [176]:
df_train.shape

(10, 17)

In [181]:
# df_train = all_values[0]
# df_test = all_values[1]


model.fit(df_train, np.array(all_labels[0]).reshape(37, 3), batch_size=12, epochs=10)




Epoch 1/10


TypeError: ignored

* <font color=blue> **Exercise 11. (1 point)** Evaluate your model on training set. The output of your neural network will be a column vector corresponding to one-hot encoding you performed earlier. Choose the column corresponding to maximum value and assign it crystal structure accordingly </font>

In [ ]:
#Enter your code here

* <font color=blue> **Exercise 12. (1 point)** Evaluate your code on test set </font>

In [ ]:
#Enter your code here

* <font color=blue>  Congrats you have trained and evaluted your first model using Keras library. Lets get more practice with this library.</font>

* <font color=blue> **Exercise 13. (5 points)** Increase the number of hidden units in your model to 50. Compile and fit this new neural network on training set. Evaluate the performance on training set as well as test set. Please note that you are fitting the model only on training set and "seperatly" evaluating the performance on training and test set  </font>

In [ ]:
#Enter your code here

* <font color=blue> **Exercise 14. (5 points)** Add another dense layer to your model. Keep the number of hidden units in both layers to be 25. Compile and fit this new neural network on training set. Evaluate the performance on training set as well as test set. </font>

In [ ]:
#Enter your code here

### Optional

<ul>
    <li> Try different parameters of the keras model and evaluate the performance on this dataset. Eg. you may want to change the number of epochs, optimization algorithm, batch size etc.</li>
    <li> Neural network training is inherently random in nature. Eg. given the initial values of the weights, models might behave differently. As we are working with a very small dataset, that difference can be small. Nevertheless, I highly recommend to read about randomness in neural network training, work out different random settings and evaluate your models. Another randomness in your results can be due to different training and test splits </li>
    <li> You can also use cross-validation to select different neural network architecture. </li>
    <li> Dropout is a method to obtain sparse neural networks. Read about this method and implement a dropout network on this data set. </li>
    </ul>
    
    